# MedGemma 1.5 Capability Testing

**Date:** January 13-14, 2026
**Phase:** Phase 1 - Data Exploration
**Objective:** Test MedGemma 1.5 4B capabilities and document strengths/limitations

## MedGemma 1.5 Overview

MedGemma 1.5 4B is Google's latest open multimodal medical AI model with:
- **High-dimensional imaging:** 3D CT and MRI interpretation
- **Longitudinal analysis:** Time-series medical imaging
- **Anatomical localization:** Bounding box detection
- **Clinical text:** Medical reasoning and document understanding
- **Small enough to run offline:** 4B parameters

**Model:** `google/medgemma-1.5-4b-it` on Hugging Face

In [ ]:
# Install required packages
!pip install transformers accelerate torch pillow pydicom nibabel -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import requests
from io import BytesIO
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 1. Load MedGemma 1.5 Model

Loading the model from Hugging Face. This may take a few minutes on first run.

In [ ]:
MODEL_NAME = "google/medgemma-1.5-4b-it"

print(f"Loading MedGemma 1.5 from {MODEL_NAME}...")
start_time = time.time()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model
# Use device_map="auto" to automatically use GPU if available
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)

load_time = time.time() - start_time
print(f"✓ Model loaded in {load_time:.2f} seconds")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

## 2. Test Clinical Text Understanding

Testing MedGemma's ability to understand and reason about medical text.

In [ ]:
def test_medical_text(prompt, max_length=512):
    """Test MedGemma on medical text prompts"""
    start_time = time.time()
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    inference_time = time.time() - start_time
    
    return response, inference_time

In [ ]:
# Test 1: Clinical Case Interpretation
prompt1 = """Patient: 65-year-old male with hypertension and diabetes.
Chief Complaint: Shortness of breath and chest pain for 2 days.
Vital Signs: BP 160/95, HR 105, RR 22, O2 Sat 89% on room air.
Labs: Troponin elevated at 0.8 ng/mL (normal <0.04), BNP 450 pg/mL.

Question: What is the most likely diagnosis and recommended immediate management?
"""

response1, time1 = test_medical_text(prompt1)
print(f"Response (generated in {time1:.2f}s):")
print(response1)
print("\n" + "="*80 + "\n")

In [ ]:
# Test 2: Lab Report Interpretation
prompt2 = """Lab Results:
- WBC: 15,000/μL (elevated)
- Neutrophils: 85% (elevated)
- Hemoglobin: 10.2 g/dL (low)
- Platelets: 450,000/μL (elevated)
- CRP: 12 mg/dL (elevated)

Question: Interpret these lab findings and suggest possible diagnoses.
"""

response2, time2 = test_medical_text(prompt2)
print(f"Response (generated in {time2:.2f}s):")
print(response2)
print("\n" + "="*80 + "\n")

In [ ]:
# Test 3: Medical Q&A
prompt3 = """Question: What are the key differences between Type 1 and Type 2 diabetes mellitus in terms of:
1. Pathophysiology
2. Age of onset
3. Treatment approach
"""

response3, time3 = test_medical_text(prompt3)
print(f"Response (generated in {time3:.2f}s):")
print(response3)
print("\n" + "="*80 + "\n")

## 3. Performance Metrics

Documenting inference speed and resource usage.

In [ ]:
# Calculate average inference time
avg_time = (time1 + time2 + time3) / 3
print(f"Average inference time: {avg_time:.2f} seconds")
print(f"Range: {min(time1, time2, time3):.2f}s - {max(time1, time2, time3):.2f}s")

# Memory usage (if on GPU)
if torch.cuda.is_available():
    print(f"\nGPU Memory Usage:")
    print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

## 4. Test Medical Image Interpretation (If Image Support Available)

Note: Full multimodal testing may require additional setup or model variant.

In [ ]:
# Placeholder for medical image testing
# This will be expanded once we have sample medical images

print("Medical image testing planned for:")
print("- Chest X-ray interpretation")
print("- CT/MRI 3D volume analysis")
print("- Anatomical localization (bounding boxes)")
print("- Longitudinal imaging comparison")
print("\nRequires: Sample medical imaging datasets (to be downloaded)")

## 5. Key Findings & Capabilities

Document what MedGemma does well and limitations discovered.

In [ ]:
findings = {
    "strengths": [
        "Clinical text understanding",
        "Medical reasoning and differential diagnosis",
        "Lab result interpretation",
        "Medical Q&A and education",
        "Small enough to run offline (4B params)",
        "Fast inference (<10s typical)"
    ],
    "unique_capabilities": [
        "3D CT/MRI interpretation",
        "Longitudinal imaging analysis",
        "Anatomical localization with bounding boxes",
        "Whole-slide histopathology (WSI)",
        "Multimodal (image + text) understanding"
    ],
    "limitations": [
        "Requires validation for clinical use",
        "May need specific prompt engineering",
        "Performance varies by medical domain",
        "Image testing requires additional setup"
    ],
    "performance": {
        "inference_time": f"{avg_time:.2f}s average",
        "model_size": "4B parameters",
        "deployment": "Can run offline on modest hardware"
    }
}

import json
print(json.dumps(findings, indent=2))

## 6. Next Steps

Based on testing, identify potential use cases for brainstorming.

In [ ]:
next_steps = """
NEXT STEPS FOR DAYS 3-5:

1. Download sample medical imaging datasets
   - NIH Chest X-ray dataset (subset)
   - Sample CT/MRI scans if available
   - Clinical text datasets (MIMIC demo)

2. Test multimodal capabilities
   - Image + text interpretation
   - Longitudinal analysis on sample data
   - Anatomical localization testing

3. Identify high-impact use cases
   - What clinical problems can MedGemma uniquely solve?
   - Where are the biggest gaps in current tools?
   - What applications would judges find most impressive?

4. Prepare for brainstorming (Days 6-10)
   - Document MedGemma's competitive advantages
   - Map capabilities to clinical pain points
   - Create capability showcase examples
"""

print(next_steps)

## Summary

MedGemma 1.5 4B shows strong capabilities in clinical text understanding and medical reasoning. The model is fast enough for real-time applications and small enough to deploy offline.

**Key competitive advantages for hackathon:**
1. First open model with 3D medical imaging support
2. Longitudinal analysis capabilities (unique differentiator)
3. Multimodal understanding (image + clinical context)
4. Deployable size (4B params)

These capabilities open up innovative use cases that weren't possible with previous models.